In [3]:
!pip3 install -qU langchain langchain_community faiss-cpu   
# segun chat gpt

In [1]:
# levanta archivo txt
with open("C:/Users/Usuario/Desktop/Rimoldi.txt") as f:
  reforma = f.read()

# print (reforma)

In [2]:
#The models used
embed_model = "nomic-embed-text"
model = "gemma2:2b"

In [23]:
# hace chunk naive  ## juego con parametros
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=50,
    length_function=len,
)

In [24]:
naive_chunks = text_splitter.split_text(reforma)

for chunk in naive_chunks[0:4]:
  print(chunk + "\n")

IdENTIdAd dE LA ExPERIENCIA REfORMIsTA PLATENsE (1918-1921)
Prof. Marcelo J. Rimoldi
        “No hay duda que nuestro tiempo es de jóvenes. El péndulo de la Historia  
siempre inquieto, asciende ahora por el cuadrante mocedado”
Ceñimos el concepto “Reforma en La Plata” al 
trance vivido por la universidad local entre 1918 
y 1921. Puede que estos límites resulten arbitrarios, por cuanto la cronología del movimien to no se circunscribe a la explosión del mismo y 
su culminación con los cambios estatutarios en 
cada universidad adherida. Hecha la salvedad,

cada universidad adherida. Hecha la salvedad, 
ese cuadrienio bastó para que, en sus moldes, la 
Universidad de La Plata asimilase y participase del 
armado y concreción de la propuesta reformista. 
Refiere Martínez Estrada, que existen no menos 
de un centenar de definiciones teóricas correctas 
de Estado (Martínez Estrada, 1956:252). Salvan do las entendibles distancias, afirmaríamos algo 
análogo respecto de la Reforma. Al sobrevol

In [25]:
# Convertir cadenas a objetos Document
from langchain.schema import Document

texts = [Document(page_content=text) for text in naive_chunks]


In [26]:
print (texts)

[Document(page_content='IdENTIdAd dE LA ExPERIENCIA REfORMIsTA PLATENsE (1918-1921)\nProf. Marcelo J. Rimoldi\n        “No hay duda que nuestro tiempo es de jóvenes. El péndulo de la Historia  \nsiempre inquieto, asciende ahora por el cuadrante mocedado”\nCeñimos el concepto “Reforma en La Plata” al \ntrance vivido por la universidad local entre 1918 \ny 1921. Puede que estos límites resulten arbitrarios, por cuanto la cronología del movimien to no se circunscribe a la explosión del mismo y \nsu culminación con los cambios estatutarios en \ncada universidad adherida. Hecha la salvedad,'), Document(page_content='cada universidad adherida. Hecha la salvedad, \nese cuadrienio bastó para que, en sus moldes, la \nUniversidad de La Plata asimilase y participase del \narmado y concreción de la propuesta reformista. \nRefiere Martínez Estrada, que existen no menos \nde un centenar de definiciones teóricas correctas \nde Estado (Martínez Estrada, 1956:252). Salvan do las entendibles distancias,

In [27]:
%%time
# Generando el contexto

from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

# Generating embeddings
embeddings = OllamaEmbeddings(model=embed_model)

# Loading the LLM from Ollama  # recordar Temperature
llm = Ollama(model=model)

# Creating FAISS index
db = FAISS.from_documents(texts, embeddings)

# Creating retriever
retriever = db.as_retriever()

In [28]:
# Invoke 
retriever.invoke("que hizo Rivarola")

[Document(page_content='de necesidades, la de Rivarola, que no supo explotar políticamente profundizando los principios con los que González había dispuesto la \nprerreforma platense, y la necesidad de un espíritu rebelde que más allá de los reclamos justos, \nno pensó en ningún momento negociar para evitar la explosión del antagonismo. Rivarola atizó \nesa rebeldía y ella se llevó por delante y aplastó \nla alternativa de una reforma civilizada, pacífica, \nsin víctimas ni vencedores.\nA partir de 1922 el espíritu de lucha parece \ndeclinar. Después de tanta acción sobrevino el'),
 Document(page_content='“presente”, los interlocutores platenses contribuyeron a la confección de la plataforma principista \nde la Reforma. Restaba el aterrizaje de ese cuerpo \nde doctrina a realidad tan particular como la platense, en la que por ejemplo, el propio Presidente \nRivarola accedía en octubre de 1918 a algunas \ndemandas estudiantiles, aunque sin conceder \nvoto a los representantes de la Fede

# pasamos a otra fase

In [29]:
# hace template

from langchain_core.prompts import ChatPromptTemplate

rag_template = """\
Utilice el siguiente contexto para responder la consulta del usuario. Si no puede responder, responda con "No lo sé".'.

consulta del usuario:
{question}

contexto:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)

In [30]:
# Arma la cadena

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context" : retriever, "question" : RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [22]:
# Envia el prompt overlap=0
%%time
rag_chain.invoke("Que hizo Rivarola")

'Rivarola fue presidente de la Universidad Nacional de La Plata y  participó en la reforma local, aunque tuvo conflictos con la FULP (Federación Universitaria de Estudiantes de La Plata). Su postura antagónica se manifestó al renunciar el 28 de junio. \n'

In [32]:
# overlap 50
%%time
rag_chain.invoke("Que hizo Rivarola")

CPU times: total: 15.6 ms
Wall time: 18.6 s


'Rivarola, como presidente de la FULP, enfrentó un conflicto con estudiantes y profesores durante una huelga. Se enfrentó a la presión de los estudiantes y profesores que reclamaban cambios en las universidades. \n\nA pesar de que Rivarola accedió a algunas demandas estudiantiles, no pudo evitar que se intensificara la tensión y que la situación llegara a un punto crítico. La muerte de un estudiante, David Francisco Viera, fue un factor clave que desencadenó una ruptura con la FULP y un conflicto con el Poder Ejecutivo Nacional.\n'

In [34]:
# overlap 50
%%time
rag_chain.invoke("Que hizo Rivarola")

CPU times: total: 0 ns
Wall time: 10.4 s


'Rivarola fue presidente de la FULP y se enfrentó a una huelga estudiantil en la Universidad de la Plata.  Su postura antagónica provocó la suspensión de exámenes y la expulsión del Consejo de la FULP, y finalmente la renuncia de Rivarola. \n'

# observo que al reiterar el invoke va variando las respuestas, aunque las veo correctas